# 📊 Crypto BI — Análisis Completo de Criptomonedas

**Proyecto:** Business Intelligence 3  
**Autores:** Juan David Reyes Cure, Julio David Suarez Olaya, Adriana Michelle Diaz Suarez

## Pipeline de Análisis

Este notebook ejecuta el pipeline completo de análisis:

1. **📥 Carga de Datos:** Dataset limpio con features derivadas
2. **🔍 Clustering:** Segmentación con K-Means, DBSCAN y Agglomerative
3. **📈 Series Temporales ARIMA:** Modelos estadísticos clásicos
4. **🧠 Redes Neuronales RNN:** Modelos LSTM/GRU con TensorFlow
5. **📊 Evaluación:** Métricas MAE, RMSE y Silhouette Score

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Imports de módulos propios (en la misma carpeta scripts/)
from clustering import kmeans_cluster, dbscan_cluster, agglomerative_cluster
from models_arima import train_arima, forecast_arima
from models_rnn import train_rnn

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Cargar datos
DATA = Path('../data/crypto_clean_BTC_ETH_BNB.csv')
df = pd.read_csv(DATA, parse_dates=['date'])

print(f"✅ Datos cargados: {len(df)} filas, {len(df.columns)} columnas")
print(f"📅 Rango: {df['date'].min()} a {df['date'].max()}")
print(f"💰 Criptomonedas: {', '.join(df['symbol'].unique())}")
df.head()

## 🔍 Clustering: Segmentación de Patrones de Mercado

Aplicamos 3 familias de algoritmos de clustering:

### 1. **K-Means** (Particionante)
- Divide los datos en k grupos predefinidos
- Minimiza la distancia intra-cluster

### 2. **DBSCAN** (Basado en Densidad)
- Detecta clusters de forma arbitraria
- Identifica outliers (ruido) automáticamente

### 3. **Agglomerative** (Jerárquico)
- Clustering bottom-up aglomerativo
- Construye jerarquía de clusters

**Features utilizadas:** `daily_return`, `roll_vol_30d`  
**Métrica de evaluación:** Silhouette Score (rango -1 a 1, mayor es mejor)

In [ ]:
# Seleccionar features para clustering
features = ['daily_return', 'roll_vol_30d']
dropna_df = df.dropna(subset=features)

print(f"📊 Datos para clustering: {len(dropna_df)} observaciones")
print(f"🎯 Features: {features}\n")

# 1. K-Means (Particionante)
print("1️⃣ K-Means Clustering...")
km_labels, km_pipe, km_sil = kmeans_cluster(dropna_df, features, n_clusters=3)
print(f"   Silhouette Score: {km_sil:.4f}")

# 2. DBSCAN (Basado en densidad)
print("\n2️⃣ DBSCAN Clustering...")
db_labels, db_pipe, db_sil = dbscan_cluster(dropna_df, features, eps=0.3, min_samples=20)
print(f"   Silhouette Score: {db_sil:.4f}")
print(f"   Outliers detectados: {(db_labels['label'] == -1).sum()}")

# 3. Agglomerative (Jerárquico)
print("\n3️⃣ Agglomerative Clustering...")
ag_labels, ag_pipe, ag_sil = agglomerative_cluster(dropna_df, features, n_clusters=3)
print(f"   Silhouette Score: {ag_sil:.4f}")

# Resumen comparativo
print("\n" + "="*50)
print("📊 RESUMEN COMPARATIVO")
print("="*50)
results = pd.DataFrame({
    'Algoritmo': ['K-Means', 'DBSCAN', 'Agglomerative'],
    'Silhouette Score': [km_sil, db_sil, ag_sil],
    'N° Clusters': [3, len(db_labels['label'].unique()) - (1 if -1 in db_labels['label'].values else 0), 3]
})
print(results.to_string(index=False))
print(f"\n🏆 Mejor algoritmo: {results.loc[results['Silhouette Score'].idxmax(), 'Algoritmo']}")

results

## 📈 Modelo ARIMA: Predicción de Precios (BTC)

**ARIMA** (AutoRegressive Integrated Moving Average) es un modelo estadístico clásico para series temporales.

### Características:
- **AR (p):** Componente autoregresivo (valores pasados)
- **I (d):** Diferenciación para estacionariedad
- **MA (q):** Media móvil de errores pasados

### Configuración:
- **Split:** 80% train / 20% test (validación temporal)
- **Order:** (1,1,1) - configurable
- **Métricas:** MAE (Mean Absolute Error) y RMSE (Root Mean Squared Error)

In [ ]:
# Filtrar datos de BTC
btc = df[df['symbol']=='BTC'][['date','price_usd']].copy()

print(f"📊 Datos BTC: {len(btc)} observaciones")
print(f"📅 Periodo: {btc['date'].min()} a {btc['date'].max()}")
print(f"\n⏳ Entrenando modelo ARIMA (puede tardar 1-2 minutos)...\n")

# Entrenar modelo ARIMA
res, y_train, y_test, preds, arima_metrics = train_arima(
    btc, 
    date_col='date', 
    target_col='price_usd',
    order=(1,1,1),  # (p,d,q)
    train_ratio=0.8
)

# Mostrar resultados
print("="*50)
print("📊 RESULTADOS ARIMA")
print("="*50)
print(f"MAE (Mean Absolute Error):  ${arima_metrics['MAE']:,.2f}")
print(f"RMSE (Root Mean Squared Error): ${arima_metrics['RMSE']:,.2f}")
print(f"\n📈 Train size: {len(y_train)} días")
print(f"📉 Test size:  {len(y_test)} días")

# Visualización simple
print(f"\n💰 Precio real promedio (test): ${y_test.mean():,.2f}")
print(f"🎯 Predicción promedio: ${preds.mean():,.2f}")
print(f"📊 Error promedio: ${arima_metrics['MAE']:,.2f} ({(arima_metrics['MAE']/y_test.mean()*100):.2f}%)")

arima_metrics

## 🧠 Modelo RNN: Redes Neuronales Recurrentes (BTC)

**LSTM** (Long Short-Term Memory) es una arquitectura de red neuronal recurrente diseñada para aprender dependencias a largo plazo.

### Características:
- **Lookback Window:** 30 días (usa 30 valores pasados para predecir el siguiente)
- **Arquitectura:** LSTM(64 unidades) → Dense(1)
- **Normalización:** MinMaxScaler (0-1)
- **Early Stopping:** Evita overfitting automáticamente

### Ventajas sobre ARIMA:
- ✅ Captura patrones no lineales
- ✅ Memoria a largo plazo
- ✅ No requiere estacionariedad

**Nota:** Usamos `epochs=5` para demostración rápida (producción: 50-100 epochs)

In [ ]:
print("🧠 Entrenando modelo LSTM...")
print("⏳ Esto puede tardar 2-3 minutos dependiendo del hardware\n")

# Entrenar modelo RNN (LSTM)
model, scaler, rnn_metrics = train_rnn(
    btc, 
    date_col='date', 
    target_col='price_usd', 
    model_type='LSTM',    # Alternativa: 'GRU'
    lookback=30,          # Ventana de 30 días
    epochs=5,             # Bajo para demo (producción: 50-100)
    batch_size=32,
    train_ratio=0.8
)

# Mostrar resultados
print("\n" + "="*50)
print("📊 RESULTADOS LSTM")
print("="*50)
print(f"MAE (Mean Absolute Error):  ${rnn_metrics['MAE']:,.2f}")
print(f"RMSE (Root Mean Squared Error): ${rnn_metrics['RMSE']:,.2f}")
print(f"\n🏆 Arquitectura: LSTM(64) → Dense(1)")
print(f"📊 Lookback window: 30 días")

# Comparación con ARIMA
print("\n" + "="*50)
print("📊 COMPARACIÓN: ARIMA vs LSTM")
print("="*50)
comparison = pd.DataFrame({
    'Modelo': ['ARIMA', 'LSTM'],
    'MAE': [arima_metrics['MAE'], rnn_metrics['MAE']],
    'RMSE': [arima_metrics['RMSE'], rnn_metrics['RMSE']]
})
print(comparison.to_string(index=False))

best_model = 'ARIMA' if arima_metrics['MAE'] < rnn_metrics['MAE'] else 'LSTM'
print(f"\n🏆 Mejor modelo (menor MAE): {best_model}")

rnn_metrics

## 📊 Visualización de Clusters

Graficamos los clusters en el espacio de features (daily_return vs roll_vol_30d)

In [ ]:
# Preparar datos para visualización
plot_df = dropna_df[features].copy()
plot_df['KMeans'] = km_labels['label'].values
plot_df['DBSCAN'] = db_labels['label'].values
plot_df['Agglomerative'] = ag_labels['label'].values

# Crear subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# K-Means
scatter1 = axes[0].scatter(plot_df['daily_return'], plot_df['roll_vol_30d'], 
                           c=plot_df['KMeans'], cmap='viridis', alpha=0.6, s=20)
axes[0].set_title(f'K-Means\nSilhouette: {km_sil:.4f}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Daily Return')
axes[0].set_ylabel('Rolling Volatility 30D')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# DBSCAN
scatter2 = axes[1].scatter(plot_df['daily_return'], plot_df['roll_vol_30d'], 
                           c=plot_df['DBSCAN'], cmap='viridis', alpha=0.6, s=20)
axes[1].set_title(f'DBSCAN\nSilhouette: {db_sil:.4f}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Daily Return')
axes[1].set_ylabel('Rolling Volatility 30D')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster (-1=Outlier)')

# Agglomerative
scatter3 = axes[2].scatter(plot_df['daily_return'], plot_df['roll_vol_30d'], 
                           c=plot_df['Agglomerative'], cmap='viridis', alpha=0.6, s=20)
axes[2].set_title(f'Agglomerative\nSilhouette: {ag_sil:.4f}', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Daily Return')
axes[2].set_ylabel('Rolling Volatility 30D')
axes[2].grid(True, alpha=0.3)
plt.colorbar(scatter3, ax=axes[2], label='Cluster')

plt.tight_layout()
plt.show()

print("✅ Visualización completada")

## 📝 Conclusiones y Resultados Finales

### 🎯 Resumen del Análisis

Este notebook demostró la implementación completa de técnicas avanzadas de Business Intelligence aplicadas a criptomonedas:

#### 1. **Clustering**
- ✅ Implementados 3 algoritmos: K-Means, DBSCAN, Agglomerative
- ✅ Evaluación con Silhouette Score
- ✅ Identificación de patrones de mercado y outliers

#### 2. **Series Temporales - ARIMA**
- ✅ Modelo estadístico clásico
- ✅ Validación temporal (80/20)
- ✅ Métricas: MAE y RMSE

#### 3. **Redes Neuronales - LSTM**
- ✅ Deep Learning con TensorFlow/Keras
- ✅ Arquitectura: LSTM(64) → Dense(1)
- ✅ Early Stopping para prevenir overfitting

### 📊 Cumplimiento de Objetivos Sprint 2

| Objetivo | Estado |
|----------|--------|
| Implementar análisis de series temporales | ✅ |
| Identificar patrones y tendencias | ✅ |
| Clusterización (densidad, particionante, jerárquico) | ✅ |
| Modelos de predicción (ARIMA, LSTM, GRU) | ✅ |
| Métricas de evaluación | ✅ |
| Módulos backend | ✅ |
| Notebook ejecutado | ✅ |

### 🚀 Próximos Pasos (Sprint 3)
- Dashboard interactivo con Streamlit
- API REST con FastAPI
- Backtesting y walk-forward validation
- Grid search para optimización de hiperparámetros